# Amharic Hate Speech Detector using Random Forest Classifier

## Preprocessing

### Importing Dataset

In [200]:
import re
import math
import pandas
import string
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [201]:
%matplotlib inline

In [202]:
DATASET_FILEPATH = "datasets/dataset.xls"
raw_data = pandas.read_excel(DATASET_FILEPATH, sheet_name='sheet')

raw_data.head()

,Comment No.,0,Comment,Like,(view source),Timestamp
0,1.0,0.0,Definitely,0,view comment,"Monday, April 20, 2020 11:29am GMT"
1,2.0,NaN,የሕዝን:: እቲ ህዘቢ ግና እስካብ መዓዝ እዩ ላተሸወደ ክነብር?,2,view comment,"Monday, April 20, 2020 11:30am GMT"
2,3.0,0.0,✔✔✔✔✔,0,view comment,"Monday, April 20, 2020 11:31am GMT"
3,4.0,0.0,Indeed!,0,view comment,"Monday, April 20, 2020 11:31am GMT"
4,5.0,0.0,real,0,view comment,"Monday, April 20, 2020 11:32am GMT"


### Extracting Data

In [203]:
labels = raw_data.iloc[:, 1]
comments = raw_data.iloc[:, 2]

### Remove Alphabets

In [204]:
comments = [re.sub(r"[A-Za-z]", "", str(comment)) for comment in comments]

### Remove Numbers

In [205]:
comments = [re.sub('[0-9]', "", str(comment)) for comment in comments]

### Remove Punctuations

In [207]:
punctuations = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~፡።፣፥¡"
comments = [str(comment).translate(str.maketrans('', '', punctuations)) for comment in comments]

### Remove Whitespaces

In [208]:
comments = [(" ".join([ word for word in str(comment).split()])) for comment in comments]

### Remove Empty Comments

In [212]:
X = []
Y = []

number_of_comments = len(comments)
for i in range(number_of_comments):
    if comments[i].strip() and not math.isnan(float(labels[i])):
        X.append(comments[i])
        Y.append(labels[i])

## Bag of Words Encoding

In [213]:
from sklearn.feature_extraction.text import CountVectorizer

In [214]:
count_vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7)
X = count_vectorizer.fit_transform(X).toarray()

In [215]:
print("Training Dataset Shape: ", X.shape)
print("Training Label Shape: ", len(Y))

Training Dataset Shape:  (2573, 601)
Training Label Shape:  2573


## Split Dataset

In [216]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [217]:
print("X_train Shape: ", X_train.shape)
print("X_test Shape: ", X_test.shape)

X_train Shape:  (2058, 601)
X_test Shape:  (515, 601)


In [218]:
print("Y_train Shape: ", len(Y_train))
print("Y_test Shape: ", len(Y_test))

Y_train Shape:  2058
Y_test Shape:  515


## Random Forest Classifier

In [219]:
from sklearn.ensemble import RandomForestClassifier

In [220]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, Y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

## Evaluate Model

In [221]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [222]:
Y_pred = classifier.predict(X_test)

### Confusion Matrix

In [223]:
print(confusion_matrix(Y_test, Y_pred))

[[493   4]
 [ 18   0]]


### Classification Report

In [224]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98       497
         1.0       0.00      0.00      0.00        18

    accuracy                           0.96       515
   macro avg       0.48      0.50      0.49       515
weighted avg       0.93      0.96      0.94       515



### Accuracy Score

In [225]:
print(accuracy_score(Y_test, Y_pred))

0.9572815533980582
